In [1]:
import requests, zipfile, io
import pandas as pd

# 1) Acquisition des données

## Récupération des données socio-économiques 

Disponibles sur le site de l'INSEE.

In [2]:
url = 'https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip'

In [3]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("INSEE/")

## Récupération des données des élections municipales de 2020

Disponibles sur le site data.gouv.

In [4]:
municipales_2020_T1 = pd.read_excel('https://static.data.gouv.fr/resources/elections-municipales-2020-liste-des-candidats-elus-au-t1-et-liste-des-communes-entierement-pourvues/20200617-160846/maires-au-17-06-2020-vf.xlsx')
municipales_2020_T2 = pd.read_excel('https://static.data.gouv.fr/resources/municipales-2020-resultats-2nd-tour/20200629-192435/2020-06-29-resultats-t2-communes-de-1000-hab-et-plus.xlsx')

## Récupération des données des élections municipales de 2014

Il s'agit de la seule élection où une liste finale des maires élus existe.

In [5]:
municipales_2014 = pd.read_excel('https://www.data.gouv.fr/storage/f/2014-06-17T12-56-58/maires-17-06-2014.xlsx')

# 2) Nettoyage des données et présentation

## Elections municipales 2020 :

In [6]:
# Renommer les colonnes du premier tour (attention code non idem-potent)
municipales_2020_T1.columns = municipales_2020_T1.iloc[0]
municipales_2020_T1.drop(labels=[0], axis=0, inplace=True)
municipales_2020_T1.reset_index(drop=True, inplace=True)

# On renomme la colonne du nom de l'élu
municipales_2020_T1.rename(columns={"Nom de l'élu": "Nom de l'élu en 2020"}, inplace=True)

In [7]:
# Ajout du code postal (en réalité code commune, mais renommé ainsi pour éviter la confusion avec 'Code de la commune')

# Ajout d'un 0 si besoin devant le numéro du département
def change_dep(dep):
    try:
        if int(dep)<10:
            return "0"+str(dep)
        return dep
    except:
        return dep

# Ajout des 0 devant le numéro du département et transition int vers string
def change_commune(com):
    try:
        if com <10:
            return "00"+str(com)
        elif com<100:
            return "0" + str(com)
        else :
            return str(com)
    except:
        return str(com)

municipales_2020_T1['Code du département (Maire)'] = municipales_2020_T1['Code du département (Maire)'].apply(lambda ligne: change_dep(ligne))
municipales_2020_T1['Code Insee de la commune'] = municipales_2020_T1['Code Insee de la commune'].apply(lambda ligne: change_commune(ligne))

# Cration de la colonne avec le code postal
municipales_2020_T1['Code postal'] = municipales_2020_T1['Code du département (Maire)'].astype(str) + municipales_2020_T1['Code Insee de la commune'].astype(str)

In [8]:
municipales_2020_T1

,Code du département (Maire),Libellé de département (Maires),Code Insee de la commune,Libellé de la commune,Nom de l'élu en 2020,Prénom de l'élu,Code sexe,Date de naissance,Code profession,Libellé de la profession,Date de début de la fonction,Code postal
0,01,AIN,001,L'Abergement-Clémenciat,BOULON,Daniel,M,1951-03-04 00:00:00,74,Anciens cadres,2020-05-26 00:00:00,01001
1,01,AIN,002,L'Abergement-de-Varey,ORSET,Max,M,1947-11-02 00:00:00,65,"Ouvriers qualifiés de la manutention, du magas...",2020-05-27 00:00:00,01002
2,01,AIN,004,Ambérieu-en-Bugey,FABRE,Daniel,M,1961-09-07 00:00:00,74,Anciens cadres,2020-05-28 00:00:00,01004
3,01,AIN,005,Ambérieux-en-Dombes,PERNET,Pierre,M,1961-07-29 00:00:00,34,"Professeurs, professions scientifiques",2020-05-27 00:00:00,01005
4,01,AIN,006,Ambléon,BIONDA,Annie,F,1951-11-28 00:00:00,77,Anciens employés,2020-05-25 00:00:00,01006
...,...,...,...,...,...,...,...,...,...,...,...,...
28382,ZP,POLYNESIE FRANCAISE,715,Faa a,TEMARU,"Oscar, Manutahi",M,1944-11-01 00:00:00,74,Anciens cadres,2020-05-23 00:00:00,ZP715
28383,ZP,POLYNESIE FRANCAISE,737,Puka Puka,VILLANT,Raphaël,M,1961-02-08 00:00:00,54,Employés administratifs d'entreprise,2020-05-25 00:00:00,ZP737
28384,ZP,POLYNESIE FRANCAISE,738,Punaauia,LISSANT,Simplicio,M,1962-08-02 00:00:00,74,Anciens cadres,2020-05-23 00:00:00,ZP738
28385,ZP,POLYNESIE FRANCAISE,741,Rapa,NARII,Tuanainai,M,1963-08-27 00:00:00,37,Cadres administratifs et commerciaux d'entreprise,2020-05-25 00:00:00,ZP741


In [9]:
# Renommer les colonnes du second tour
nom_cols_T2 = list(municipales_2020_T2.columns)

for i in range(30,78):
    nom_cols_T2[i] = nom_cols_T2[i-12]
    
for i in range(30,78):
    nom_cols_T2[i] = nom_cols_T2[i] + "." + str(int((i-18)/12))
    
municipales_2020_T2.columns = nom_cols_T2

In [10]:
# Ajout du code postal (en réalité code commune, mais renommé ainsi pour éviter la confusion avec 'Code de la commune')
municipales_2020_T2['Code postal'] = municipales_2020_T2['Code du département'] + municipales_2020_T2['Code de la commune']

In [11]:
municipales_2020_T2

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,Nom.4,Prénom.4,Liste.4,Sièges / Elu.4,Sièges Secteur.4,Sièges CC.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Code postal
0,01,Ain,034,Belley,5757,2877,49.97,2880,50.03,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01034
1,01,Ain,160,Ferney-Voltaire,3798,2280,60.03,1518,39.97,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01160
2,01,Ain,194,Jassans-Riottier,4357,2689,61.72,1668,38.28,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01194
3,01,Ain,249,Miribel,6847,4064,59.35,2783,40.65,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01249
4,01,Ain,354,Saint-Genis-Pouilly,5015,3108,61.97,1907,38.03,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,ZP,Polynésie française,728,Maupiti,1085,54,4.98,1031,95.02,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP728
1425,ZP,Polynésie française,733,Paea,9180,2771,30.19,6409,69.81,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP733
1426,ZP,Polynésie française,734,Papara,8597,3290,38.27,5307,61.73,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP734
1427,ZP,Polynésie française,735,Papeete,18056,7989,44.25,10067,55.75,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZP735


## Elections municipales de 2014 :

In [12]:
# Renommer les colonnes (attention code non idem-potent)
municipales_2014.columns = municipales_2014.iloc[1]
municipales_2014.drop(labels=[0,1], axis=0, inplace=True)
municipales_2014.reset_index(drop=True, inplace=True)

# On renomme la colonne du nom de l'élu
municipales_2014.rename(columns={"Nom de l'élu": "Nom de l'élu en 2014"}, inplace=True)

In [13]:
municipales_2014['Code du département (Maire)'] = municipales_2014['Code du département (Maire)'].apply(lambda ligne: change_dep(ligne))
municipales_2014['Code Insee de la commune'] = municipales_2014['Code Insee de la commune'].apply(lambda ligne: change_commune(ligne))

# Cration de la colonne avec le code postal
municipales_2014['Code postal'] = municipales_2014['Code du département (Maire)'].astype(str) + municipales_2014['Code Insee de la commune'].astype(str)

In [14]:
municipales_2014

1,Code du département (Maire),Libellé de département (Maires),Code Insee de la commune,Libellé de la commune,Population de la commune,Nom de l'élu en 2014,Prénom de l'élu,Code sexe,Date de naissance,Code profession,Libellé de la profession,Code postal
0,01,AIN,001,L'Abergement-Clémenciat,780,BOULON,Daniel,M,1951-03-04 00:00:00,61,Retraités salariés privés,01001
1,01,AIN,002,L'Abergement-de-Varey,234,ORSET,Max,M,1947-11-02 00:00:00,65,Autres retraités,01002
2,01,AIN,004,Ambérieu-en-Bugey,13839,FABRE,Daniel,M,1961-09-07 00:00:00,51,Cadres (entreprises publiques),01004
3,01,AIN,005,Ambérieux-en-Dombes,1600,PERNET,Pierre,M,1961-07-29 00:00:00,42,Professeurs du secondaire et techn.,01005
4,01,AIN,006,Ambléon,112,BIONDA,Annie,F,1951-11-28 00:00:00,63,Retraités fonct.publique (sf enseig.),01006
...,...,...,...,...,...,...,...,...,...,...,...,...
36623,ZP,POLYNESIE FRANCAISE,056,Ua-Huka,621,OHU,Nestor,M,1962-04-02 00:00:00,52,Employés (autres entrep. publiques),ZP056
36624,ZP,POLYNESIE FRANCAISE,057,Ua-Pou,2173,KAIHA,Joseph,M,1960-12-07 00:00:00,43,Enseignants 1er deg.-directeurs école,ZP057
36625,ZP,POLYNESIE FRANCAISE,058,Uturoa,3697,TEROOATEA,Sylviane,F,1966-10-13 00:00:00,50,Cadres sup. (entreprises publiques),ZP058
36626,ZS,SAINT PIERRE ET MIQUELON,501,Miquelon-Langlade,635,DE LIZARRAGA,Jean,M,1943-12-17 00:00:00,63,Retraités fonct.publique (sf enseig.),ZS501


# 3) Détermination des maires de 2020

## Second tour des élections municipales de 2020

Le base de données du premier tour tour fournit directement les maires élus. Cependant, dans celle du second tour, il n'y a que les scores de chaque candidat. Il faut donc déterminer les maires élus au decond tour.

In [15]:
# Combien y a-t-il de candidats au second tour ?
def compte_candidats(liste):
    return len(liste) - sum(pd.isna(liste).astype(int))

In [16]:
communes = municipales_2020_T2['Code postal'].unique()

noms = [f'Nom.{i}' for i in range(1,5)]
noms.insert(0, 'Nom')

voix = [f'Voix.{i}' for i in range(1,5)]
voix.insert(0, 'Voix')

muni2 = municipales_2020_T2.copy(deep=True)

In [17]:
maires = pd.DataFrame(columns=['Code postal', "Nom de l'élu en 2020"])

for i in communes:
    nombre_candidats = compte_candidats(list(muni2.loc[muni2['Code postal']==i][noms].iloc[0]))   
    max_voix = 0
    indice_max = 0
    if nombre_candidats == 1:
        maires.loc[len(maires)] = [i, muni2.loc[muni2['Code postal']==i]['Nom'].unique()]
    else:    
        for j in range(nombre_candidats):
            actuel = muni2.loc[muni2['Code postal']==i][voix[j]].sum()
            if actuel > max_voix:
                max_voix = actuel
                indice_max = j
        maires.loc[len(maires)] = [i, muni2.loc[muni2['Code postal']==i][noms[indice_max]].unique()]

On observe une singularité pour les communes de moins de 1 000 habitants. Pour ces communes, l'élection des conseillers municipaux se déroule au scrutin majoritaire plurinominal à deux tours, avec panachage. La somme des voix de tous les candidats peut donc être supérieure au nombre de votants.

Nous allons donc ne pas pouvoir analyser ces communes.

In [18]:
# Rendons les noms des maires plus lisibles en les mettant au format string
def normalise_nom(nom):
    return str(nom)[2:-2]

maires["Nom de l'élu en 2020"] = maires["Nom de l'élu en 2020"].apply(lambda ligne: normalise_nom(ligne))

In [19]:
maires

,Code postal,Nom de l'élu en 2020
0,01034,LAHUERTA
1,01160,RAPHOZ
2,01194,REVERCHON
3,01249,GAITET
4,01354,BERTRAND
...,...,...
1424,ZP728,RAUFAUORE
1425,ZP733,GEROS
1426,ZP734,PUNUA
1427,ZP735,BUILLARD


## Agrégation des deux tours pour obtenir tous maires élus en 2020

In [20]:
municipales_2020 = pd.concat([municipales_2020_T1[['Code postal', "Nom de l'élu en 2020"]], maires], axis=0)

In [21]:
municipales_2020

,Code postal,Nom de l'élu en 2020
0,01001,BOULON
1,01002,ORSET
2,01004,FABRE
3,01005,PERNET
4,01006,BIONDA
...,...,...
1424,ZP728,RAUFAUORE
1425,ZP733,GEROS
1426,ZP734,PUNUA
1427,ZP735,BUILLARD


## Agrégation des résultats des années 2014 et 2020

Faisons l'aggrégation des maires élus en 2014 et 2020 par ville.
Attention : il existe des données manquantes car son également incluses certains villes de moins de 1000 habitants. Cela sera mis à jour plus tard avec la dernière base de données.

In [22]:
municipales_2014_et_2020 = municipales_2020.copy(deep= True)
municipales_2014_et_2020 = municipales_2014_et_2020.merge(municipales_2014[["Code postal", "Nom de l'élu en 2014", 'Libellé de la commune']], on = 'Code postal')

In [23]:
municipales_2014_et_2020

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune
0,01001,BOULON,BOULON,L'Abergement-Clémenciat
1,01002,ORSET,ORSET,L'Abergement-de-Varey
2,01004,FABRE,FABRE,Ambérieu-en-Bugey
3,01005,PERNET,PERNET,Ambérieux-en-Dombes
4,01006,BIONDA,BIONDA,Ambléon
...,...,...,...,...
29641,ZN824,DEDANE,DALAP,Pouébo
29642,ZN826,TIDJINE EP HMAE,TIDJINE EP HMAE,Poum
29643,ZN827,GORO ATU,KURTOVITCH EP METZDORF,Poya
29644,ZN831,BOATATE KOLEKOLE,DOUNEHOTE,Voh


# 4) Maires candidats à leurs successions 

Déterminons si les maires de 2014 ont tenté de se faire réélire en 2020. Pour cela on utilise la base de données de data.gouv qui comprend tous les candidats aux élections municipales de 2020.

In [24]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps
candidats_2020 = pd.read_excel('https://static.data.gouv.fr/resources/elections-municipales-2020-candidatures-au-1er-tour/20200304-105133/livre-des-listes-et-candidats.xlsx')

In [25]:
# Renommer les colonnes (attention code non idem-potent)
candidats_2020.columns = candidats_2020.iloc[1]
candidats_2020.drop(labels=[0,1], axis=0, inplace=True)
candidats_2020.reset_index(drop=True, inplace=True)

In [26]:
candidats_2020['Code du département'] = candidats_2020['Code du département'].apply(lambda ligne: change_dep(ligne))
candidats_2020['Code commune'] = candidats_2020['Code commune'].apply(lambda ligne: change_commune(ligne))

# Cration de la colonne avec le code postal
candidats_2020['Code postal'] = candidats_2020['Code du département'].astype(str) + candidats_2020['Code commune'].astype(str)

In [27]:
# Attention, l'éxécution de cette cellule prend plusieurs minutes ! 

municipales_2014_et_2020['Tentative de réélection'] = True
for cpt, maire in enumerate(municipales_2014_et_2020["Nom de l'élu en 2014"]):
    if maire != municipales_2014_et_2020["Nom de l'élu en 2020"].iloc[cpt]:
        ville = municipales_2014_et_2020['Code postal'].iloc[cpt]
        municipales_2014_et_2020['Tentative de réélection'].iloc[cpt] = maire in candidats_2020.loc[candidats_2020['Code postal']==ville]['Nom candidat']
        if len(candidats_2020.loc[candidats_2020['Code postal']==ville]['Nom candidat'])<1:
            municipales_2014_et_2020['Tentative de réélection'].iloc[cpt] = 'Ville <1000'

/Users/henri/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [28]:
municipales_2014_et_2020

,Code postal,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection
0,01001,BOULON,BOULON,L'Abergement-Clémenciat,True
1,01002,ORSET,ORSET,L'Abergement-de-Varey,True
2,01004,FABRE,FABRE,Ambérieu-en-Bugey,True
3,01005,PERNET,PERNET,Ambérieux-en-Dombes,True
4,01006,BIONDA,BIONDA,Ambléon,True
...,...,...,...,...,...
29641,ZN824,DEDANE,DALAP,Pouébo,False
29642,ZN826,TIDJINE EP HMAE,TIDJINE EP HMAE,Poum,True
29643,ZN827,GORO ATU,KURTOVITCH EP METZDORF,Poya,False
29644,ZN831,BOATATE KOLEKOLE,DOUNEHOTE,Voh,False


'Ville <1000' indique que le maire d'une ville de moins de 1000 habitants ne s'est pas fait réélire. Les maires des villes de moins de 1000 habitants s'étant fait réélire sont compris dans 'True' et non pas 'Ville <1000'.

# 5) Fusion avec les données de l'INSEE

L'INSEE utilise un "Code géographique" (colonne CODGEO) pour identifier les communes, que nous n'avons pas pour les communes dont le maire a été élu au premier tour.
Afin de fusionner les bases de données socio-économiques et des élections, nous avons donc besoin de récupérer ce code géographique.

In [29]:
meta_insee = pd.read_csv(filepath_or_buffer='INSEE/meta_dossier_complet.csv', sep=';', encoding='l5')
insee = pd.read_csv(filepath_or_buffer='INSEE/dossier_complet.csv', sep=';', encoding='l5')

/Users/henri/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
meta_insee = meta_insee[['COD_MOD', 'LIB_MOD']].dropna()
meta_insee.reset_index(drop=True, inplace=True)

In [31]:
def problème_FR(phrase):
    return phrase.replace('Ã©', 'é').replace('Ã«', 'ë').replace('Ã¨', 'è').replace('Ã\x89', 'É').replace('Ã´', 'ô').replace('Ã§', 'ç').replace('Ã®', 'î').replace('Ã\x8b', 'Ë').replace('Ã\x88', 'È').replace('Ã¯', 'ï')

In [32]:
meta_insee['LIB_MOD'] = meta_insee['LIB_MOD'].apply(lambda ligne: problème_FR(ligne))
meta_insee.columns = ['CODGEO', 'Libellé de la commune']

In [33]:
meta_insee

,CODGEO,Libellé de la commune
0,01001,L'Abergement-Clémenciat
1,01002,L'Abergement-de-Varey
2,01004,Ambérieu-en-Bugey
3,01005,Ambérieux-en-Dombes
4,01006,Ambléon
...,...,...
34995,75116,Paris 16e Arrondissement
34996,75117,Paris 17e Arrondissement
34997,75118,Paris 18e Arrondissement
34998,75119,Paris 19e Arrondissement


Fusionnons avec les résultats des élections municipales

In [36]:
municipales_2014_et_2020 = municipales_2014_et_2020.merge(meta_insee, on = 'Libellé de la commune').drop(columns=['Code postal'])
municipales_2014_et_2020.drop_duplicates(subset=['Nom de l\'élu en 2014', 'Nom de l\'élu en 2020', 'Libellé de la commune'], inplace=True)

In [37]:
municipales_2014_et_2020

,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection,CODGEO
0,BOULON,BOULON,L'Abergement-Clémenciat,True,01001
1,ORSET,ORSET,L'Abergement-de-Varey,True,01002
2,FABRE,FABRE,Ambérieu-en-Bugey,True,01004
3,PERNET,PERNET,Ambérieux-en-Dombes,True,01005
4,BIONDA,BIONDA,Ambléon,True,01006
...,...,...,...,...,...
34312,MIRANVILLE,MIRANVILLE,La Possession,True,97408
34313,DOMEN,ROBERT,Saint-Leu,False,97413
34314,THIEN-AH-KOON,THIEN AH KOON,Le Tampon,False,97422
34315,PAUSÉ,PAUSÉ,Les Trois-Bassins,True,97423


Nous pouvons maintenant ajouter aux données de l'INSEE, les données relatives aux élections.

In [41]:
insee

,CODGEO,P19_POP,P19_POP0014,P19_POP1529,P19_POP3044,P19_POP4559,P19_POP6074,P19_POP7589,P19_POP90P,P19_POPH,...,CPGEO522,VV22,VVUH22,VVLIT22,RT22,RTUH22,RTLIT22,AJCS22,AJCSUH22,AJCSLIT22
0,1001,779.0,158.508629,102.468339,131.523972,194.112147,131.050309,56.014471,5.322133,397.885214,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1002,256.0,58.646351,29.609476,74.028483,38.684019,35.381928,17.582020,2.067723,129.517516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1004,14134.0,2642.167045,3076.783602,2730.398468,2391.347109,2054.139468,1044.007592,195.156716,6790.643609,...,0.0,0.0,0.0,0.0,1.0,16.0,16.0,0.0,0.0,0.0
3,1005,1751.0,346.859147,268.250842,362.580808,380.267677,272.181257,111.034231,9.826038,877.465208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1006,112.0,12.218182,16.290909,15.272727,29.527273,27.490909,10.181818,1.018182,63.127273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,97613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34996,97614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34997,97615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34998,97616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
new = insee.merge(municipales_2014_et_2020, on='CODGEO')
new

,CODGEO,P19_POP,P19_POP0014,P19_POP1529,P19_POP3044,P19_POP4559,P19_POP6074,P19_POP7589,P19_POP90P,P19_POPH,...,RT22,RTUH22,RTLIT22,AJCS22,AJCSUH22,AJCSLIT22,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection
0,28377,1761.0,318.775971,222.662944,331.945621,413.950655,320.837060,144.730914,8.096834,915.917503,...,0.0,0.0,0.0,0.0,0.0,0.0,GALLAND,MAITRE,Sorel-Moussel,False
1,28379,467.0,94.864702,42.518564,94.981717,116.671755,84.342075,29.538328,4.082858,227.838660,...,0.0,0.0,0.0,0.0,0.0,0.0,MOLET,MOLET,Soulaires,True
2,28380,1998.0,440.065895,319.620047,438.965507,421.464746,280.042069,94.718408,3.123329,1008.108177,...,0.0,0.0,0.0,0.0,0.0,0.0,PLAULT,PLAULT,Sours,True
3,28382,889.0,184.691342,132.258750,173.422604,200.609627,116.606912,66.090688,15.320078,448.327779,...,0.0,0.0,0.0,0.0,0.0,0.0,PELLEGRIN,PELLEGRIN,Terminiers,True
4,28383,706.0,156.000000,93.000000,148.000000,162.000000,107.000000,32.000000,8.000000,361.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,PENTHER,AUBERT DU PETIT TOUARD,Theuville,Ville <1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,30343,604.0,103.331351,51.839867,113.333001,145.142993,110.212993,61.963313,18.176483,303.208138,...,0.0,0.0,0.0,0.0,0.0,0.0,PESENTI,CHAMPETIER,Verfeuil,Ville <1000
909,30345,343.0,37.394489,27.800274,39.947775,78.462495,112.630034,30.888852,15.876081,167.491389,...,0.0,0.0,0.0,0.0,0.0,0.0,MIAILLE,PAMBOUTZOGLOU,La Vernarède,Ville <1000
910,30346,1794.0,312.000000,216.000000,323.000000,387.000000,394.000000,148.000000,14.000000,876.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,SAUZET,SAUZET,Vers-Pont-du-Gard,True
911,30347,1379.0,266.647162,202.440512,280.634003,281.160238,263.415070,80.593613,4.109403,671.036841,...,0.0,0.0,0.0,0.0,0.0,0.0,LAURENT,LAURENT,Vestric-et-Candiac,True


In [43]:
municipales_2014_et_2020.loc[municipales_2014_et_2020['Libellé de la commune']=='Cilaos']

,Nom de l'élu en 2020,Nom de l'élu en 2014,Libellé de la commune,Tentative de réélection,CODGEO
34316,TÉCHER,TECHER,Cilaos,False,97424


A cause de ce problème de non correspondance des accents, nous allons devoir enlever tous les accents.